In [76]:
import os
import json
import natsort
import datetime
import pandas
import re

# We will determine which result files are relevant for analysis.
result_list = []
for f in natsort.natsorted(os.listdir('../client/results')):
    try:
        fp = open('../client/results/' + f, 'r')
        time_string = re.search(r'(graphix|neo4j)_(([0-9]|T)+).json', f).group(2)
        try:
            j = json.loads(fp.readline())
            result_list += [[
                j['clientClass'].split('BenchmarkClient')[0],
                datetime.datetime.strptime(time_string, "%Y%m%dT%H%M%S"),
                j['executionID'],
                j['config']['notes']
            ]]
        except json.JSONDecodeError:
            pass
    except IsADirectoryError:
        pass

pandas.set_option('display.max_colwidth', None)
pandas.DataFrame(result_list, columns=['System', 'Time', 'Execution ID', 'Notes'])

,System,Time,Execution ID,Notes
0,Graphix,2023-02-28 20:06:14,98ab0355-c945-45e6-a86f-58fb5535cb30,Using CBO. EXPLAIN issued before query.
1,Graphix,2023-02-28 22:56:02,04f5fb88-f7f5-4075-9890-37291c859767,Using CBO. EXPLAIN issued before query.
2,Graphix,2023-03-01 03:23:45,9268fcd6-8a61-40cb-b01f-107105a1dd65,Using CBO. EXPLAIN issued before query.
3,Graphix,2023-03-01 22:25:10,5b8a6d7a-6e57-4b06-83d2-b7c8ec533bad,Using CBO. EXPLAIN issued before query.
4,Graphix,2023-03-02 18:44:51,f5046266-da89-4b63-bb4a-1e4a2d2852fd,1NC. Using CBO. CC on NC. Cluster built w/ ansible.
5,Graphix,2023-04-20 03:38:11,ad01f0ed-fb37-486a-b020-ee5084e01fce,Just BI workload. First run.
6,Graphix,2023-04-20 03:56:50,d1ab0c50-ccb9-480f-b84d-fd1213f5148e,Just BI workload. Second run.
7,Graphix,2023-04-20 04:35:11,27e99a18-6017-466f-ac72-c16af8fd4d6b,Just BI workload. Debug run (issue with BI2).
8,Graphix,2023-04-20 04:41:03,3031c0f8-0f30-4359-ba3d-7b9f21c95877,Just BI workload. Debug run (issue with BI2).
9,Graphix,2023-04-20 04:47:17,2741338d-55be-4741-82d9-31090a49d46c,Just BI workload. Debug run (issue with BI2).


In [77]:
# Define the whitelist here, using the dataframe above! We start with our initial Graphix runs...
initial_graphix_executions = [
    'e6cd282c-7da9-4875-9542-698b6538395e',
    '2316e26c-050f-45f0-b777-411ef291837e',
    'bf9db62c-8a95-4e22-9c2f-a147d6353c4f',
    'dc0e21b9-f598-4c01-befb-c1ef227f7feb',
    'b6444e6e-4523-40eb-aa30-946303b2d0d5',
    'a1ff52f1-557b-45e1-8d2b-f3b60d614a81'
]
execution_disjuncts = list(
    f'R.executionID = "{e}" AND '
    f'R.queryFile NOT LIKE "bi-18%.sqlpp" AND '
    f'R.queryFile NOT LIKE "bi-7%.sqlpp" AND '
    f'R.queryFile NOT LIKE "bi-5.sqlpp"'
    for e in initial_graphix_executions
)

# Now, our next Neo4J runs...
neo4j_executions = [
    '23750fbb-5cb6-407b-b503-a0e78b50d2ce',
    '1dcc51cd-d3cf-4802-8318-f7850f196e1f',
    'ba8fa58f-397d-4e59-ade6-935deff05e22'
]
execution_disjuncts += list(
    f'R.executionID = "{e}"'
    for e in neo4j_executions
)

# Now, our followup Graphix runs...
followup_graphix_executions = [
    '8fdd2cb7-6c75-4180-b1e0-4efb3185a88c',
    'ce693dd4-f8d3-4960-89f1-b5dd0ed5021a',
    'cb4aafdf-2ec8-41b9-b98b-1ff0a975f747',
    '0a326ea1-03ff-41c0-b3ea-eb51e9393bda',
    '529ef238-8af4-437a-b83e-a099ac37c359',
    '2f531ec4-f8c7-4a2f-aa90-5e7ceb2e59f6'
]
execution_disjuncts += list(
    f'R.executionID = "{e}"'
    for e in followup_graphix_executions
)
execution_disjuncts

['R.executionID = "e6cd282c-7da9-4875-9542-698b6538395e" AND R.queryFile NOT LIKE "bi-18%.sqlpp" AND R.queryFile NOT LIKE "bi-7%.sqlpp" AND R.queryFile NOT LIKE "bi-5.sqlpp"',
 'R.executionID = "2316e26c-050f-45f0-b777-411ef291837e" AND R.queryFile NOT LIKE "bi-18%.sqlpp" AND R.queryFile NOT LIKE "bi-7%.sqlpp" AND R.queryFile NOT LIKE "bi-5.sqlpp"',
 'R.executionID = "bf9db62c-8a95-4e22-9c2f-a147d6353c4f" AND R.queryFile NOT LIKE "bi-18%.sqlpp" AND R.queryFile NOT LIKE "bi-7%.sqlpp" AND R.queryFile NOT LIKE "bi-5.sqlpp"',
 'R.executionID = "dc0e21b9-f598-4c01-befb-c1ef227f7feb" AND R.queryFile NOT LIKE "bi-18%.sqlpp" AND R.queryFile NOT LIKE "bi-7%.sqlpp" AND R.queryFile NOT LIKE "bi-5.sqlpp"',
 'R.executionID = "b6444e6e-4523-40eb-aa30-946303b2d0d5" AND R.queryFile NOT LIKE "bi-18%.sqlpp" AND R.queryFile NOT LIKE "bi-7%.sqlpp" AND R.queryFile NOT LIKE "bi-5.sqlpp"',
 'R.executionID = "a1ff52f1-557b-45e1-8d2b-f3b60d614a81" AND R.queryFile NOT LIKE "bi-18%.sqlpp" AND R.queryFile NOT LIK

In [78]:
import requests
import os

requests.post('http://localhost:19002/query/service', {
    'statement': f"""
        DROP   DATAVERSE SNB.Analysis IF EXISTS;
        CREATE DATAVERSE SNB.Analysis;
        USE    SNB.Analysis;

        CREATE TYPE GenericType AS {{ }};
        CREATE EXTERNAL DATASET RawResults (GenericType)
        USING  localfs ( ("path"="localhost://{os.path.abspath('../client/results')}"),
                 ("expression"="(.)*results.(graphix|neo4j)_([0-9]|T)+.json$"),
                 ("format"="json") );

        CREATE VIEW ProcessedResults AS
            SELECT R.*,
                   queryClass,
                   queryVersion,
                   clusterSize,
                   scaleFactor,
                   system,
                   CASE WHEN IS_STRING(R.results)
                        THEN GREATEST(LEN(SPLIT(R.results, "{{")) - 1, 0)
                        ELSE LEN(R.results)
                        END AS resultCount,
                   ROW_NUMBER() OVER (PARTITION BY scaleFactor, queryClass, queryVersion, system, clusterSize
                                      ORDER BY     R.logTime) AS rowNumber
            FROM   SNB.Analysis.RawResults R
            LET    queryClass   = SPLIT(SPLIT(R.queryFile, ".")[0], "-v")[0],
                   queryVersion = SPLIT(SPLIT(R.queryFile, ".")[0], "-v")[1],
                   sfString     = SPLIT(SPLIT(R.config.parametersDir, "/mnt/efs/fs1/sf-")[1], "/parameters")[0],
                   scaleFactor  = CASE sfString
                                       WHEN "0p3"
                                       THEN "0.3"
                                       ELSE sfString
                                       END,
                   clusterSize  = CASE R.clientClass
                                  WHEN "Neo4jBenchmarkClient"
                                  THEN NULL
                                  ELSE LTRIM(SUBSTR(R.config.notes, POSITION(R.config.notes, "-node cluster") - 2, 2))
                                  END,
                   system       = CASE R.clientClass
                                       WHEN "Neo4jBenchmarkClient"
                                       THEN "Neo4J"
                                       ELSE RTRIM(R.clientClass, "BenchmarkClient")
                                       END
            WHERE  ( {' OR '.join("(" + e + ")" for e in execution_disjuncts)} ) AND
                   ( EVERY p IN [queryClass, scaleFactor, system] SATISFIES p IS NOT UNKNOWN END );

        CREATE VIEW Statistics AS
            SELECT   R.queryClass,
                     R.queryVersion,
                     R.scaleFactor,
                     R.system,
                     R.clusterSize,
                     ( SELECT DISTINCT g.R.config.notes,
                                       g.R.executionID
                       FROM   g ) AS notes,
                     ( SELECT VALUE g.R.clientTime
                       FROM   g ) AS allTimes,
                     STDDEV_POP(R.clientTime) AS spreadTime,
                     MAX(R.clientTime) AS maxTime,
                     MIN(R.clientTime) AS minTime,
                     AVG(R.clientTime) AS avgTime,
                     COUNT(*) AS numPoints
            FROM     SNB.Analysis.ProcessedResults R
            WHERE    R.rowNumber > 3
            GROUP BY R.scaleFactor,
                     R.queryClass,
                     R.queryVersion,
                     R.system,
                     R.clusterSize
            GROUP AS g;
    """
}).json()

{'requestID': 'f2c35ef1-9f44-4547-9fab-9d98ddd194d6',
 'plans': {},
 'status': 'success',
 'metrics': {'elapsedTime': '1.191393125s',
  'executionTime': '1.069182625s',
  'compileTime': '0ns',
  'resultCount': 0,
  'resultSize': 0,
  'processedObjects': 0}}

In [ ]:
import plotly.offline

plotly.offline.init_notebook_mode()

In [73]:
import requests
import pandas
import plotly.graph_objects
import plotly.colors
import natsort

# Fetch our data.
response = requests.post('http://localhost:19002/query/service', {
    'statement': f"""
        SET    `compiler.sortmemory` "128MB";
        SET    `compiler.groupmemory` "128MB";
        SELECT CASE WHEN LEN(stringTimes) < 10
                    THEN STRING_JOIN(stringTimes, "<br>")
                    ELSE STRING_JOIN(stringTimes, "<br>") || "<br>..."
                    END AS timesAsString,
               S.* EXCLUDE notes
        FROM   SNB.Analysis.Statistics S
        LET    stringTimes = (
                   SELECT VALUE TO_STRING(ROUND(T, 3))
                   FROM   S.allTimes T
                   LIMIT  10
               )
    """
}).json()
df = pandas.json_normalize(response['results'])

# Plot our results with a bar chart.
fig = plotly.graph_objects.Figure()
query_classes = natsort.natsorted(df.queryClass.unique())
for i, qc in enumerate(query_classes):
    for j, system in enumerate(['Graphix', 'Neo4J']):
        if system == 'Neo4J':
            qc_sf = df.query(
                f'system == "{system}" and '
                f'queryClass == "{qc}"'
            )
        else:
            qc_sf = df.query(
                f'system == "{system}" and '
                f'queryClass == "{qc}" and '
                f'clusterSize == "1"'
            )
        fig.add_trace(plotly.graph_objects.Bar(
            x=qc_sf.scaleFactor,
            y=qc_sf.avgTime,
            name=system,
            alignmentgroup=system,
            texttemplate='%{y:.3r}',
            textposition='outside',
            visible=i == 0,
            marker={'color': plotly.colors.qualitative.D3[j]},
            customdata=qc_sf[['spreadTime', 'maxTime', 'minTime', 'timesAsString']],
            hovertemplate="<b>Average:</b> %{y} +/-%{customdata[0]:.3f}<br>"
                          "<b>Maximum:</b> %{customdata[1]:.3f}<br>"
                          "<b>Minimum:</b> %{customdata[2]:.3f}<br>"
                          "<b>Using:</b><br>%{customdata[3]}"
        ))
fig.update_layout(
    title={
        'text': f'Graphix vs. Neo4J: {query_classes[0]}',
        'xanchor': 'center',
        'yanchor': 'top',
        'x': 0.5,
        'font': {
            'size': 24
        }
    },
    xaxis={
        'title': 'Scale Factor',
        'type': 'category',
        'categoryorder': 'array',
        'categoryarray': sorted([float(f) for f in df.scaleFactor.unique()])
    },
    yaxis={
        'title': 'Running Time (s)',
        'showspikes': True,
    }
)

# For all query classes, build an update action.
buttons = list()
for i, qc in enumerate(query_classes):
    visible_data_vector = [False for f in range(0, len(query_classes) * 2)]
    for j in range(2):
        visible_data_vector[(i * 2) + j] = True
    buttons.append({
        'label': qc,
        'method': 'update',
        'args': [
            {'visible': visible_data_vector},
            {'title': f'Graphix vs. Neo4J: {qc}'}
        ]
    })
fig.update_layout(updatemenus=[{
    'active': 0,
    'buttons': buttons
}])
fig.show()

In [90]:
import requests
import pandas
import plotly.graph_objects
import plotly.colors
import natsort

# Fetch our data.
response = requests.post('http://localhost:19002/query/service', {
    'statement': f"""
        SET    `compiler.sortmemory` "128MB";
        SET    `compiler.groupmemory` "128MB";
        SELECT CASE WHEN LEN(stringTimes) < 10
                    THEN STRING_JOIN(stringTimes, "<br>")
                    ELSE STRING_JOIN(stringTimes, "<br>") || "<br>..."
                    END AS timesAsString,
               S.* EXCLUDE notes
        FROM   SNB.Analysis.Statistics S
        LET    stringTimes = (
                   SELECT VALUE TO_STRING(ROUND(T, 3))
                   FROM   S.allTimes T
                   LIMIT  10
               );
    """
}).json()
df = pandas.json_normalize(response['results'])

# In addition to our plot, we also want to build CSV outputs.
output_neo4j, output_graphix = [], []

# Plot our results with a bar chart.
fig = plotly.graph_objects.Figure()
cluster_sizes = [1, 2, 4, 8, 16, 32]
query_classes = natsort.natsorted(df.queryClass.unique())
neo4j_visible_vector = []
for i, qc in enumerate(query_classes):
    qc_sf = df.query(
        f'system == "Neo4J" and '
        f'queryClass == "{qc}" and '
        f'scaleFactor == "300"'
    )
    neo4j_has_value = len(qc_sf.avgTime) > 0
    neo4j_visible_vector += [neo4j_has_value]
    neo4j_value = qc_sf.avgTime.item() if neo4j_has_value else None
    fig.add_shape(
        type='line',
        yref='y',
        y0=(neo4j_value if neo4j_has_value else 0),
        y1=(neo4j_value if neo4j_has_value else 0),
        xref='paper',
        x0=0,
        x1=1,
        visible=i == 0,
        line={
            'color': plotly.colors.qualitative.D3[-1],
            'width': 3,
            'dash': 'dot'
        }
    )
    if neo4j_has_value:
        output_neo4j += [f'{neo4j_value},"{qc}",-1000']
        output_neo4j += [f'{neo4j_value},"{qc}",1000']
    for j, cluster_size in enumerate(cluster_sizes):
        if qc == 'bi-7':
            version_conjunct = 'queryVersion == "2" and '
        elif qc == 'bi-18':
            version_conjunct = 'queryVersion == "2" and '
        else:
            version_conjunct = ''
        qc_sf = df.query(
            f'system == "Graphix" and '
            f'clusterSize == "{cluster_size}" and '
            f'queryClass == "{qc}" and '
            f'{version_conjunct} '
            f'scaleFactor == "300"'
        )
        if len(qc_sf.avgTime) > 0:
            output_graphix += [f'{qc_sf.avgTime.item()},{cluster_size},"{qc}"']
        fig.add_trace(plotly.graph_objects.Bar(
            x=[cluster_size],
            y=qc_sf.avgTime,
            name=f'{cluster_size} Nodes',
            texttemplate='%{y:.3r}',
            visible=i == 0,
            marker={'color': plotly.colors.qualitative.D3[j]},
            customdata=qc_sf[['spreadTime', 'maxTime', 'minTime', 'timesAsString']],
            hovertemplate="<b>Average:</b> %{y} +/-%{customdata[0]:.3f}<br>"
                          "<b>Maximum:</b> %{customdata[1]:.3f}<br>"
                          "<b>Minimum:</b> %{customdata[2]:.3f}<br>"
                          "<b>Using:</b><br>%{customdata[3]}"
        ))

# Write our results to a CSV.
with open('results/neo4j.csv', 'w') as fp:
    fp.write('time,queryclass,misc\n')
    for p in output_neo4j:
        fp.write(p + '\n')
with open('results/graphix.csv', 'w') as fp:
    fp.write('time,clustersize,queryclass\n')
    for p in output_graphix:
        fp.write(p + '\n')

# For all query classes, build an update action.
fig.update_layout(
    title={
        'text': f'Graphix Scale-Out: {query_classes[0]}',
        'xanchor': 'center',
        'yanchor': 'top',
        'x': 0.5,
        'font': {
            'size': 24
        }
    },
    xaxis={
        'title': 'Cluster Size',
        'type': 'category',
        'categoryorder': 'array',
        'categoryarray': cluster_sizes
    },
    yaxis={
        'title': 'Running Time (s)',
        'showspikes': True,
    }
)
buttons = list()
for i, qc in enumerate(query_classes):
    visible_data_vector = [False for f in range(0, len(query_classes) * len(cluster_sizes))]
    for j in range(len(cluster_sizes)):
        visible_data_vector[(i * len(cluster_sizes)) + j] = True
    visible_shapes_dict = {f'shapes[{j}].visible': False for j in range(len(query_classes))}
    if neo4j_visible_vector[i]:
        visible_shapes_dict[f'shapes[{i}].visible'] = True
    buttons.append({
        'label': qc,
        'method': 'update',
        'args': [
            {'visible': visible_data_vector},
            {'title.text': f'Graphix Scale-Out: {qc}', **visible_shapes_dict}
        ]
    })
fig.update_layout(updatemenus=[{
    'active': 0,
    'buttons': buttons
}])
fig.show()